# LaQuacco 🍅🍅🍅

## Laboratory Quality Control

### Module Imports

In [ ]:
import os
import platform
import sys
import time
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import scipy as sp
import laquacco as laq  # custom functions

### User Input

In [ ]:
# define search pattern for positive controls (if available)
control_dir = r"./tests/Polaris"
control_ext = "*.tif"
control_exc = ""
control_sub = True

# define search pattern for QC samples
sample_dir = r"./tests/Polaris"
sample_ext = "*.tif"
sample_exc = ""
sample_sub = True

# output settings
sort_chans = False  # sort alphabetically

# render channel images of outliers
show_img = False

### Check samples

In [ ]:
# list positive controls
controls = sorted(
    laq.get_files(
        path=control_dir,
        pat=control_ext,
        anti=control_exc,
        recurse=control_sub,
    ),
    key=str.lower,
)
controls_len = len(controls)
print(f"Found {controls_len} controls in {os.path.abspath(control_dir)}:")
for control in controls:
    print(f"{control.replace(control_dir, '.')}")
print()

# list samples
samples = sorted(
    laq.get_files(
        path=sample_dir,
        pat=sample_ext,
        anti=sample_exc,
        recurse=sample_sub,
    ),
    key=str.lower,
)
samples_len = len(samples)
print(f"Found {samples_len} samples in {os.path.abspath(sample_dir)}:")
for sample in samples:
    print(f"{sample.replace(sample_dir, '.')}")

### Sample Data

In [ ]:
# get control statistics
controls_img_data = {}
start = time.time()
for count, control in enumerate(controls, start=1):
    print(f"CONTROL: [{count}/{controls_len}]  {os.path.basename(control)}", end="", flush=True)
    controls_img_data[control] = laq.stats_img_data(laq.get_tiff(control))  # signal above zero
    print(f"  (-{laq.get_time_left(start, count, controls_len)})", flush=True)
print()

# prepare dictionary (set) of unique channel labels
chans = dict()  # dictionary retains insertion order
for img_data in controls_img_data.values():
    for chan in img_data:
        if chan not in ["metadata"]:
            chans[chan] = None
chans = list(chans.keys())
if sort_chans:
    chans = sorted(chans, key=str.lower)

# prepare bottom and top percentiles for analysis
chans_stats = {chan: (np.nan, np.nan) for chan in chans}
for chan in chans:
    print(f"{chan}:", flush=True)
    mean = laq.get_chan_data(controls_img_data, chan, "mean")
    minmax = laq.get_chan_data(controls_img_data, chan, "minmax", 2)
    chans_stats[chan] = (np.nanmean(mean), np.nanmean(minmax[:, 0]), np.nanmean(minmax[:, 1]))
    print(
        f"\tMaximum: {chans_stats[chan][2]:7.1f} (mean), {np.nanstd(minmax[:, 1]):7.1f} (std)\n",
        f"\tMean:    {chans_stats[chan][2]:7.1f} (mean), {np.nanstd(mean):7.1f} (std)\n",
        f"\tMinimum: {chans_stats[chan][1]:7.1f} (mean), {np.nanstd(minmax[:, 0]):7.1f} (std)\n",
    )

# get sample statistics
samples_img_data = {}
start = time.time()
for count, sample in enumerate(samples, start=1):
    print(f"SAMPLE: [{count}/{samples_len}]  {os.path.basename(sample)}", end="", flush=True)
    samples_img_data[sample] = laq.stats_img_data(laq.get_tiff(sample), chans_stats)
    print(f"  (-{laq.get_time_left(start, count, samples_len)})", flush=True)
print()

# sort sample image data and list by time stamp
samples_img_data = dict(
    sorted(samples_img_data.items(), key=lambda v: v[1]["metadata"]["date_time"])
)
samples = [sample for sample in samples_img_data.keys()]

### Graphics Settings

In [ ]:
# prepare figure dimensions
dpi = plt.rcParams["figure.dpi"]
min_pixw, min_pixh = 1600, 1200
min_width, min_height = min_pixw / dpi, min_pixh / dpi
plt.rcParams["figure.figsize"] = [min_width, min_height]

### Data Plots I - Distribution Chart

In [ ]:
print("Plotting distribution chart...\n", flush=True)

# get mean data for plots
means = []
fig, ax = plt.subplots()
for c, chan in enumerate(chans):
    mean = laq.get_chan_data(samples_img_data, chan, "mean")
    mean = mean[~np.isnan(mean)] # omit missing data
    means.append(mean)

# create violin plot
vp = ax.violinplot(means, showmeans=True, showextrema=False)
for p in vp["bodies"]:
    p.set_facecolor("black")
    p.set_edgecolor("black")
for l in ["cmeans"]:
    vp[l].set_edgecolor("dimgray")

# adjust axes
ax.set_xticks([c for c in range(1, len(chans) + 1)], labels=chans, rotation=90)
plt.ylim(bottom=0.0)

# add legend
legend = plt.legend(
    [vp["bodies"][0]],
    ["mean"],
)

# show plot
plt.tight_layout()
plt.show()

### Data Plots II - Levey-Jennings Charts

In [ ]:
print("Plotting Levey-Jennings charts...\n", flush=True)

# positions in plot
for position, sample in enumerate(samples, 1):
    print(f"{position} = {os.path.basename(sample)}")
print()

# prepare variables
slice_ext = samples_len - 1  # extend slice to either sides
fit_trend = False  # fit a linear regression model of the mean
slice_size = min(samples_len, 2 * slice_ext + 1)
assert (
    slice_size > 3
), "Zero degrees of freedom to estimate the standard deviation from the trend line."
chan_means = {}
chan_extras = {}
x_vals = range(1, samples_len + 1)
np_nan = np.full(samples_len, np.nan)

# create error bar plots
for c, chan in enumerate(chans):
    # prepare variables
    run = {stat: np_nan.copy() for stat in ["slice", "means", "stdevs"]}
    trend = {stat: np_nan.copy() for stat in ["slice", "vals", "stdevs"]}

    # get image statistics
    means = laq.get_chan_data(samples_img_data, chan, "mean")
    stdevs = laq.get_chan_data(samples_img_data, chan, "stdev")
    stderrs = laq.get_chan_data(samples_img_data, chan, "stderr")
    chan_means[chan] = means

    # get trend statistics
    if fit_trend:
        slope, inter = np.polyfit(x_vals, means, deg=1)
        trend["vals"] = slope * x_vals + inter
    else:
        trend["vals"].fill(np.nanmean(means))

    # get running statistics
    for i, mean in enumerate(means):
        run["slice"] = laq.get_run_slice(means, i, slice_ext)
        if run["slice"].size == slice_size:
            run["means"][i] = np.nanmean(run["slice"])
            run["stdevs"][i] = np.nanmean(laq.get_run_slice(stdevs, i, slice_ext))
            trend["slice"] = laq.get_run_slice(trend["vals"], i, slice_ext)
            trend["stdevs"][i] = np.nanstd(run["slice"], ddof=3)  # estimated: slope, intercept, and mean

    # get extrema from trend line
    chan_extras_keys = [("p2stdev", "m2stdev"), ("p1stdev", "m1stdev")]
    chan_extras[chan] = {
        chan_extras_keys[0][0]: trend["vals"] + 2.0 * trend["stdevs"],
        chan_extras_keys[1][0]: trend["vals"] + 1.0 * trend["stdevs"],
        chan_extras_keys[1][1]: trend["vals"] - 1.0 * trend["stdevs"],
        chan_extras_keys[0][1]: trend["vals"] - 2.0 * trend["stdevs"],
    }

    # plot statistics
    for dist in [2.0, 1.0, -1.0, -2.0]:
        linestyle = (0, (1, 2))
        if abs(dist) == 2.0:
            linestyle = (0, (1, 4))
        plt.plot(
            x_vals,
            run["means"] + dist * run["stdevs"],
            color="black",
            linewidth=1,
            linestyle=linestyle,
        )
    for upper, lower in chan_extras_keys:
        plt.fill_between(
            x_vals,
            chan_extras[chan][upper],
            chan_extras[chan][lower],
            color="black",
            alpha=0.1,
        )
    plt.plot(x_vals, trend["vals"], color="black", linewidth=1, linestyle="solid")
    plt.plot(x_vals, run["means"], color="black", linewidth=1, linestyle="dashed")

    # plot y values with errors
    plt.errorbar(
        x_vals,
        means,
        yerr=stderrs,
        fmt="o-",
        linewidth=1,
        markersize=2,
        color="black",
        label=chan,
    )

    # adjust axes
    plt.ylim(0.0)

    # add legend
    legend = plt.legend()

    # show plot
    plt.tight_layout()
    plt.show()

### Data Plots II - Extreme Values

In [ ]:
# list all images with extreme values per channel
for c, chan in enumerate(chans):
    print(f"\n{chan}:", flush=True)
    outliers = []
    for m, mean in enumerate(chan_means[chan]):
        if mean > chan_extras[chan]["p2stdev"][m]:
            outliers.append(("▲▲ ", m, samples[m], chan, mean))
        elif mean > chan_extras[chan]["p1stdev"][m]:
            outliers.append(("▲  ", m, samples[m], chan, mean))
        elif mean < chan_extras[chan]["m2stdev"][m]:
            outliers.append(("▼▼ ", m, samples[m], chan, mean))
        elif mean < chan_extras[chan]["m1stdev"][m]:
            outliers.append(("▼  ", m, samples[m], chan, mean))
    # show color bar at top
    cmap = mpl.cm.nipy_spectral
    norm = mpl.colors.Normalize(
        vmin=np.nanmedian(chans_stats[chan][0]), vmax=np.nanmedian(chans_stats[chan][1])
    )
    scalarmappable = mpl.cm.ScalarMappable(norm=norm, cmap=cmap)
    scalarmappable.set_array([])
    #fig = plt.figure(figsize=(min_width, 1))
    #ax = fig.add_axes([0.0, 0.0, 1, 0.5])
    #cbar = fig.colorbar(scalarmappable, cax=ax, orientation="horizontal")
    #plt.show()
    if outliers:
        # print list of outliers with optional channel images
        for indicator, position, sample, channel, mean in outliers:
            print(
                f"\n\t{indicator} {position} = {os.path.basename(sample)}"
                # f"  ({mean}) [{chans_minmax[chan][0]}-{chans_minmax[chan][1]}]"
            )
            if show_img:
                plt.imshow(
                    laq.get_chan_img(sample, channel),
                    cmap="nipy_spectral",
                    vmin=chans_stats[chan][0],
                    vmax=chans_stats[chan][1],
                    resample=False,
                )
                plt.show()
    else:
        print(f"\t►  (none)")

### Data Plots III - C-Band Charts

In [ ]:
print("Plotting C-Band charts...\n", flush=True)

x_vals = range(1, samples_len + 1)
for c, chan in enumerate(chans):
    print(f"\n{chan}:", flush=True)
    percs = laq.get_chan_data(samples_img_data, chan, "perc")
    bands = zip(*laq.get_chan_data(samples_img_data, chan, "bands", 4))
    
    # create line plot
    fig, ax = plt.subplots()
    for band in bands:
        has_data = np.count_nonzero(~np.isnan(band))
        if has_data:
            # get statistics
            mean = np.nanmean(band) if band else np.nan  # can't handle `np.nan`-only arrays
            stdev = np.nanstd(band) if band else np.nan
            # get extrema from trend line
            chan_extras_keys = [("p2stdev", "m2stdev"), ("p1stdev", "m1stdev")]
            chan_extras[chan] = {
                chan_extras_keys[0][0]: mean + 2.0 * stdev,
                chan_extras_keys[1][0]: mean + 1.0 * stdev,
                chan_extras_keys[1][1]: mean - 1.0 * stdev,
                chan_extras_keys[0][1]: mean - 2.0 * stdev,
            }
            # plot statistics
            for upper, lower in chan_extras_keys:
                plt.fill_between(
                    x_vals,
                    chan_extras[chan][upper],
                    chan_extras[chan][lower],
                    color="black",
                    alpha=0.1,
                )
            # plot y values
            plt.errorbar(
                x_vals,
                band,
                fmt="o-",
                linewidth=1,
                markersize=2,
                color="black",
                label=chan,
            )
    
    #plt.plot(bands, color="red", linestyle="solid")
    plt.axhline(y=chans_stats[chan][0], color="black", linestyle="dashed")
    plt.axhline(y=chans_stats[chan][0] + 1/3 * (chans_stats[chan][2] - chans_stats[chan][0]), color="black", linestyle="dotted")
    plt.axhline(y=chans_stats[chan][0] + 2/3 * (chans_stats[chan][2] - chans_stats[chan][0]), color="black", linestyle="dotted")
    plt.axhline(y=chans_stats[chan][2], color="black", linestyle="dotted")
    # add legend
    legend = plt.legend()

    # show plot
    plt.tight_layout()
    plt.show()

### Data Plots III - Ranked bands

In [ ]:
# rank images by the bands' signal-to-noise ratios

